In [1]:
import datetime
import geopandas
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2023-05-04 20:47:49.081542: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# TODO try only with best performing numeric columns for input into model first
# TODO select features for deep learning model after ranking features with random forest (feature importance)
# TODO lstm network
# TODO SMOTE algorithm, for upsampling havested fields; we only have ~300 :(

In [3]:
#local scripts
from scripts import veg_indices, utilities, plots
#from scripts.utilities import *

In [4]:
# cpied from learning_about-data.ipynb
BANDS_DICT = {   'B2': 'Blue',
            'B3': 'Green',
            'B4': 'Red',
            'B5': 'Red_Edge_1',
            'B6': 'Red_Edge_2',
            'B7': 'Red_Edge_3',
            'B8': 'NIR',
            'B8A': 'Red_Edge_4',
            'B11': 'SWIR_1',
            'B12': 'SWIR_2'}

BANDS = list(BANDS_DICT.keys())

In [5]:
# cpied from learning_about-data.ipynb
DF = geopandas.read_file('../data/merged_images.geojson')
DF.rename(columns = {'is_within_period':'har_evnt'}, inplace = True)
NUM_SAMPLES = len(np.unique(DF.image_idx)) - 1

In [230]:
DF

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,NDVI,finHarvDat,lat,lon,point_idx,start_date,end_date,har_evnt,image_idx,geometry
0,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,2022-08-11,50.655048,25.458684,p0,2022-01-08,2022-01-28,False,i0,POINT (25.45868 50.65505)
1,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,NaT,50.294633,36.289471,p1,2022-01-08,2022-01-28,False,i0,POINT (36.28947 50.29463)
2,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,NaT,50.860849,32.548484,p2,2022-01-08,2022-01-28,False,i0,POINT (32.54848 50.86085)
3,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,NaT,50.928689,31.637558,p3,2022-01-08,2022-01-28,False,i0,POINT (31.63756 50.92869)
4,0.1424,0.1083,0.030,0.0432,0.0631,0.07150,0.07760,0.08510,0.1066,0.1043,0.256335,2022-07-23,48.307964,37.348245,p4,2022-01-08,2022-01-28,False,i0,POINT (37.34824 48.30796)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,0.2557,0.1875,0.065,0.0740,0.0973,0.11695,0.14255,0.15525,0.1920,0.1847,0.327342,NaT,49.723156,36.817274,p553,2022-12-10,2022-12-30,False,i16,POINT (36.81727 49.72316)
9482,0.1616,0.1235,0.028,0.0442,0.0525,0.06390,0.08950,0.09310,0.1152,0.1073,0.373882,NaT,45.578351,28.700480,p554,2022-12-10,2022-12-30,False,i16,POINT (28.70048 45.57835)
9483,0.1675,0.1030,0.043,0.0678,0.0460,0.10960,0.23980,0.26430,0.2832,0.2783,0.720535,NaT,46.693510,35.051003,p555,2022-12-10,2022-12-30,False,i16,POINT (35.05100 46.69351)
9484,0.1000,0.1000,0.100,0.1000,0.1000,0.10000,0.10000,0.10000,0.1000,0.1000,0.000000,2022-07-30,50.451929,33.668204,p556,2022-12-10,2022-12-30,False,i16,POINT (33.66820 50.45193)


In [228]:
# cpied from learning_about-data.ipynb
df = DF.copy()
df = df[(df.NDVI) != 0] # drop invalid points
VEG_INDICES_NAMES = veg_indices.add_veg_indices(df) + ['NDVI'] 
VEG_DIFF_NAMES = veg_indices.add_veg_diff(df, VEG_INDICES_NAMES)
NUMERIC_COLS = BANDS + VEG_INDICES_NAMES + VEG_DIFF_NAMES
df = df.dropna(subset=['sample_idx']) # removes the first image, because sample_idx is NaN there


""" 
# only look at samples 6, 7, 8, 9
new_df = None
for i in range(6, 9 + 1):

    curr_df = df[df["sample_idx"] == f"s{i}"]
    if(type(new_df) == type(None)):
        new_df = curr_df
    else:
        new_df = pd.concat([new_df, curr_df])
df = new_df
"""

print(df.columns, df.shape)


# For each 3-week image, standarize each column
df = utilities.get_rm_outlier_standarize(df, NUMERIC_COLS, rm_outliers=False)

Added:  ['DVI', 'RVI', 'ARVI', 'PSSRa', 'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre', 'NIRv', 'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI']
Added:  ['sample_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff', 'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_diff', 'CIr_diff', 'MTCI_diff', 'NDVIre_diff', 'NIRv_diff', 'EVI_diff', 'NDTI_diff', 'NDMI_diff', 'MSI_diff', 'GCI_diff', 'NBRI_diff', 'BSI_diff', 'NDWI_diff', 'NDSI_diff', 'NDVI_diff']
Index(['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'NDVI',
       'finHarvDat', 'lat', 'lon', 'point_idx', 'start_date', 'end_date',
       'har_evnt', 'image_idx', 'geometry', 'DVI', 'RVI', 'ARVI', 'PSSRa',
       'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre', 'NIRv',
       'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI',
       'pt_idx', 'img_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff',
       'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_d

In [229]:
df

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,...,NDTI_diff,NDMI_diff,MSI_diff,GCI_diff,NBRI_diff,BSI_diff,NDWI_diff,NDSI_diff,NDVI_diff,sample_idx
560,-1.733345,-1.218163,2.538801,2.565978,2.582734,2.652148,2.583266,2.535112,2.496266,2.521000,...,0.000192,2.354683,-1.860549,-0.290191,1.705637,-1.810909,2.025189,2.149293,-2.144416,s0
562,0.091251,-0.030531,-0.465189,-0.464070,-0.425066,-0.491252,-0.582446,-0.605294,-0.581443,-0.624426,...,0.310313,-0.636401,0.591765,-0.564848,0.072627,0.642642,-0.459362,-0.630151,0.216412,s0
563,-0.072429,0.107892,-0.500069,-0.508641,-0.495575,-0.544710,-0.658926,-0.675985,-0.673982,-0.693985,...,0.090693,-0.705486,0.964065,-0.757482,-0.094059,0.974112,-0.657753,-0.786087,0.373323,s0
564,-0.475597,-0.137859,2.456404,2.440304,2.439523,2.404264,2.361241,2.361878,2.338312,2.323263,...,0.554492,1.747677,-1.632557,-0.370456,1.624033,-1.573719,1.942476,1.756096,-1.934649,s0
565,0.883804,0.933417,-0.442766,-0.456764,-0.437304,-0.435597,-0.451143,-0.456309,-0.456197,-0.459937,...,-0.159611,-0.547000,0.575539,-0.129763,-0.353626,0.643998,-0.627675,-0.578638,0.521988,s0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9475,3.322104,2.328019,0.132010,0.231196,0.095625,0.472431,1.725177,2.015223,2.066392,2.180961,...,-4.826355,0.732779,-1.169280,1.612652,-0.915200,-1.344209,0.282752,0.795201,0.566191,s15
9478,-1.628924,-1.062433,3.578395,3.457746,3.404489,3.259756,3.083942,2.963168,3.031307,2.897218,...,-0.668641,2.848363,-2.258848,-0.110247,1.915134,-2.202332,2.572595,2.716314,-2.818478,s15
9481,1.843392,1.368079,-0.275171,-0.257938,-0.181384,-0.168046,-0.179112,-0.158634,-0.057645,-0.060312,...,0.114312,-0.434865,0.208224,0.134505,-0.290227,0.266190,-0.606048,-0.316902,0.445377,s15
9482,-0.009429,-0.141409,-0.468817,-0.419537,-0.416868,-0.447121,-0.460629,-0.493758,-0.450153,-0.492917,...,0.411511,-0.449486,0.316610,0.088229,-0.345191,0.438725,-0.761268,-0.373365,0.573503,s15


In [205]:
def get_one_hot(df:pd.DataFrame, colName: str, retArrays=False):
    lookupTable, indexed_dataSet = np.unique(df[colName], return_inverse=True)
    tpls = tf.keras.utils.to_categorical(indexed_dataSet)

    arrays = tpls.T[1:] # remove one column (not needed)
    arrays = arrays.astype(np.float32) # tensorflow does not support bools, so float 32 is the best we can do :/
    if(retArrays):
        return arrays

    return arrays.T # return to n by m (n tuples of size m)




In [206]:
df

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,...,NDTI_diff,NDMI_diff,MSI_diff,GCI_diff,NBRI_diff,BSI_diff,NDWI_diff,NDSI_diff,NDVI_diff,sample_idx
3906,-0.870086,-0.894463,-0.362898,-0.562899,-0.712486,-0.803480,-0.029281,0.751010,0.946631,0.729749,...,-1.478007,1.603634,-1.360793,2.700617,-2.968218,-1.523347,0.521851,1.005915,1.573112,s6
3907,-0.312600,-0.369832,-0.086144,0.280409,-0.099855,0.208097,0.629703,0.362093,0.061773,0.152029,...,0.003610,0.370216,-0.424915,-0.590995,0.906889,-0.333211,1.112289,0.629439,-0.610115,s6
3908,0.709577,0.053909,-0.152589,-0.109988,-0.493141,-0.196534,1.214141,1.359916,1.234850,1.216424,...,-1.587707,-0.081406,-0.394088,0.704432,-0.719594,-0.579386,-0.135283,0.200484,1.265219,s6
3909,-0.561720,-0.722576,-0.312302,-0.473593,-0.729585,-0.671262,0.668467,1.463120,2.055841,1.519354,...,-2.537627,1.919438,-1.531427,3.167865,-3.512676,-1.735957,0.806501,1.165113,1.692973,s6
3910,-0.299037,-0.194955,-0.324494,-0.356218,-0.293845,-0.373203,-0.754462,-0.892654,-0.904964,-0.916228,...,0.557096,-0.683240,0.500220,-0.312166,0.231992,0.551830,-0.896480,-0.580907,0.203664,s6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6133,-1.223664,-1.653289,-0.874174,-0.848826,-1.376933,-0.955374,1.352084,1.767256,1.749997,1.694362,...,-1.565219,1.862842,-1.964314,2.941633,-3.306606,-2.374764,0.411842,2.136983,2.313166,s9
6134,0.585052,0.995327,-0.201453,-0.167603,0.086507,-0.184328,-0.739681,-0.749316,-0.777399,-0.679339,...,0.142589,-0.838315,1.168225,-0.907196,0.397156,1.137787,-0.727334,-1.122512,-0.399597,s9
6135,-0.186510,0.046045,-0.259642,-0.345125,0.100141,-0.173940,-0.687242,-0.689586,-0.657617,-0.606040,...,0.554367,-0.367829,0.398082,-0.382788,0.256747,0.560347,-0.615972,-0.461928,-0.459774,s9
6136,0.421999,1.102268,-0.817404,-1.045111,-0.845186,-1.091576,-1.444705,-1.206955,-1.128188,-1.259767,...,-0.706420,-1.028375,1.619866,-0.555706,-0.749974,1.448490,-2.587140,-1.076003,0.980113,s9


In [207]:


def convert_to_1hot(categorical_col_name:str):
    names = []
    arrays = get_one_hot(df, categorical_col_name, retArrays=True)
    for i in range(len(arrays)):
        arr = arrays[i]
        name = f'{categorical_col_name}_1hot_{i}'
        df[name] = arr
        names.append(name)
    return names

one_hot_names_X = convert_to_1hot("sample_idx")
one_hot_names_Y = convert_to_1hot("har_evnt")

print(df.columns)
print(one_hot_names_X)
print(one_hot_names_Y)


Index(['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'NDVI',
       'finHarvDat', 'lat', 'lon', 'point_idx', 'start_date', 'end_date',
       'har_evnt', 'image_idx', 'geometry', 'DVI', 'RVI', 'ARVI', 'PSSRa',
       'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre', 'NIRv',
       'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI',
       'pt_idx', 'img_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff',
       'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_diff', 'CIr_diff',
       'MTCI_diff', 'NDVIre_diff', 'NIRv_diff', 'EVI_diff', 'NDTI_diff',
       'NDMI_diff', 'MSI_diff', 'GCI_diff', 'NBRI_diff', 'BSI_diff',
       'NDWI_diff', 'NDSI_diff', 'NDVI_diff', 'sample_idx',
       'sample_idx_1hot_0', 'sample_idx_1hot_1', 'sample_idx_1hot_2',
       'har_evnt_1hot_0'],
      dtype='object')
['sample_idx_1hot_0', 'sample_idx_1hot_1', 'sample_idx_1hot_2']
['har_evnt_1hot_0']


In [222]:
sorted(df['point_idx'].unique(), key= lambda x: int(x[1:]))

['p0',
 'p1',
 'p2',
 'p3',
 'p4',
 'p5',
 'p6',
 'p7',
 'p8',
 'p9',
 'p10',
 'p11',
 'p12',
 'p13',
 'p14',
 'p15',
 'p16',
 'p17',
 'p18',
 'p19',
 'p20',
 'p21',
 'p22',
 'p23',
 'p24',
 'p25',
 'p26',
 'p27',
 'p28',
 'p29',
 'p30',
 'p31',
 'p32',
 'p33',
 'p34',
 'p35',
 'p36',
 'p37',
 'p38',
 'p39',
 'p40',
 'p41',
 'p42',
 'p43',
 'p44',
 'p45',
 'p46',
 'p47',
 'p48',
 'p49',
 'p50',
 'p51',
 'p52',
 'p53',
 'p54',
 'p55',
 'p56',
 'p57',
 'p58',
 'p59',
 'p60',
 'p61',
 'p62',
 'p63',
 'p64',
 'p65',
 'p66',
 'p67',
 'p68',
 'p69',
 'p70',
 'p71',
 'p72',
 'p73',
 'p74',
 'p75',
 'p76',
 'p77',
 'p78',
 'p79',
 'p80',
 'p81',
 'p82',
 'p83',
 'p84',
 'p85',
 'p86',
 'p87',
 'p88',
 'p89',
 'p90',
 'p91',
 'p92',
 'p93',
 'p94',
 'p95',
 'p96',
 'p97',
 'p98',
 'p99',
 'p100',
 'p101',
 'p102',
 'p103',
 'p104',
 'p105',
 'p106',
 'p107',
 'p108',
 'p109',
 'p110',
 'p111',
 'p112',
 'p113',
 'p114',
 'p115',
 'p116',
 'p117',
 'p118',
 'p119',
 'p120',
 'p121',
 'p122',
 'p

In [225]:
df

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,...,NBRI_diff,BSI_diff,NDWI_diff,NDSI_diff,NDVI_diff,sample_idx,sample_idx_1hot_0,sample_idx_1hot_1,sample_idx_1hot_2,har_evnt_1hot_0
3906,-0.870086,-0.894463,-0.362898,-0.562899,-0.712486,-0.803480,-0.029281,0.751010,0.946631,0.729749,...,-2.968218,-1.523347,0.521851,1.005915,1.573112,s6,0.0,0.0,0.0,0.0
3907,-0.312600,-0.369832,-0.086144,0.280409,-0.099855,0.208097,0.629703,0.362093,0.061773,0.152029,...,0.906889,-0.333211,1.112289,0.629439,-0.610115,s6,0.0,0.0,0.0,0.0
3908,0.709577,0.053909,-0.152589,-0.109988,-0.493141,-0.196534,1.214141,1.359916,1.234850,1.216424,...,-0.719594,-0.579386,-0.135283,0.200484,1.265219,s6,0.0,0.0,0.0,0.0
3909,-0.561720,-0.722576,-0.312302,-0.473593,-0.729585,-0.671262,0.668467,1.463120,2.055841,1.519354,...,-3.512676,-1.735957,0.806501,1.165113,1.692973,s6,0.0,0.0,0.0,0.0
3910,-0.299037,-0.194955,-0.324494,-0.356218,-0.293845,-0.373203,-0.754462,-0.892654,-0.904964,-0.916228,...,0.231992,0.551830,-0.896480,-0.580907,0.203664,s6,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6133,-1.223664,-1.653289,-0.874174,-0.848826,-1.376933,-0.955374,1.352084,1.767256,1.749997,1.694362,...,-3.306606,-2.374764,0.411842,2.136983,2.313166,s9,0.0,0.0,1.0,0.0
6134,0.585052,0.995327,-0.201453,-0.167603,0.086507,-0.184328,-0.739681,-0.749316,-0.777399,-0.679339,...,0.397156,1.137787,-0.727334,-1.122512,-0.399597,s9,0.0,0.0,1.0,0.0
6135,-0.186510,0.046045,-0.259642,-0.345125,0.100141,-0.173940,-0.687242,-0.689586,-0.657617,-0.606040,...,0.256747,0.560347,-0.615972,-0.461928,-0.459774,s9,0.0,0.0,1.0,0.0
6136,0.421999,1.102268,-0.817404,-1.045111,-0.845186,-1.091576,-1.444705,-1.206955,-1.128188,-1.259767,...,-0.749974,1.448490,-2.587140,-1.076003,0.980113,s9,0.0,0.0,1.0,0.0


In [223]:
#  reshape the data into a 3D tensor with shape X (num_samples, sequence_length, num_features) to feed it into a 1D CNN in TensorFlow for training.
# sequence_length: each 3-week sample
# num_samples: num of points, aka farms

points_dfs_X = []
points_dfs_Y = []

for p in sorted(df['point_idx'].unique(), key= lambda x: int(x[1:])):
    curr_df = df[df["point_idx"] == p]
    print(curr_df.shape)
    if(curr_df.shape[0] == 4): #  TODO fix this, even method1 is not done correctly :/ 
        """
        TODO
        use one of the follwoing(currently using method 1):
            1-Remove the rows with missing values.
            2-Fill in missing values with a constant
            3-Interpolate missing values
            4-Use imputation methods
        """
        curr_df_X = (curr_df[one_hot_names_X + BANDS]).to_numpy()
        points_dfs_X.append(curr_df_X) 

        curr_df_Y = (curr_df[one_hot_names_Y]).to_numpy()
        points_dfs_Y.append(curr_df_Y) 




(4, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(4, 70)
(4, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(3, 70)
(3, 70)
(4, 70)
(4, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(3, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(4, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(4, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(3, 70)
(4, 70)
(3, 70)
(3, 70)
(3, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(2, 70)
(3, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(4, 70)
(3, 70)
(3, 70)
(3, 70)
(4, 70)
(3, 70)


In [224]:
curr_df

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,...,NBRI_diff,BSI_diff,NDWI_diff,NDSI_diff,NDVI_diff,sample_idx,sample_idx_1hot_0,sample_idx_1hot_1,sample_idx_1hot_2,har_evnt_1hot_0
4463,0.753119,0.641316,-0.212329,-0.171226,-0.158228,-0.093951,0.024988,-0.081143,-0.080699,-0.206632,...,-0.013575,0.607005,-1.117648,-0.653725,0.550355,s6,0.0,0.0,0.0,0.0
5021,0.080242,-0.205906,-0.272849,-0.285226,-0.706196,-0.449542,0.527871,0.564223,0.577029,0.496656,...,-0.749229,-0.645731,-0.328488,0.509959,1.463451,s7,1.0,0.0,0.0,0.0
5579,-0.417185,-0.272877,-0.455992,-0.445247,-0.745832,-0.453823,0.228387,0.245941,0.352068,0.204620,...,-1.745090,-0.895212,-0.536235,1.144411,1.712407,s8,0.0,1.0,0.0,0.0
6137,-0.608432,-0.292357,-0.669803,-0.638109,-0.802010,-0.760304,-0.322105,-0.291678,-0.341051,-0.349493,...,-0.491515,-0.017271,-0.655980,0.251211,0.849334,s9,0.0,0.0,1.0,0.0


In [184]:
X = np.array(points_dfs_X)
Y = np.array(points_dfs_Y)

In [185]:
X.shape

(430, 4, 13)

In [186]:
Y.shape

(430, 4, 1)

In [ ]:
Y

In [187]:
Y = Y.reshape((430, 4))
Y

array([[0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [188]:
X.shape[1], X.shape[2]

(4, 13)

In [218]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization

model = tf.keras.Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(4, 13)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    BatchNormalization(),
    Dense(units=64, activation='relu'),
    Dense(units=256, activation='relu'),
    Dense(units=256, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=4)
])

#model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [219]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


In [220]:
model.fit(X, Y, epochs=200, validation_split=0.1)


Epoch 1/200
13/13 [==============================] - 2s 28ms/step - loss: 0.1411 - accuracy: 0.3566 - val_loss: 0.1068 - val_accuracy: 0.3023
Epoch 2/200
13/13 [==============================] - 0s 8ms/step - loss: 0.1111 - accuracy: 0.4289 - val_loss: 0.1056 - val_accuracy: 0.3488
Epoch 3/200
13/13 [==============================] - 0s 10ms/step - loss: 0.1055 - accuracy: 0.4315 - val_loss: 0.1053 - val_accuracy: 0.3256
Epoch 4/200
13/13 [==============================] - 0s 8ms/step - loss: 0.0978 - accuracy: 0.4496 - val_loss: 0.1055 - val_accuracy: 0.3256
Epoch 5/200
13/13 [==============================] - 0s 9ms/step - loss: 0.0928 - accuracy: 0.4574 - val_loss: 0.0984 - val_accuracy: 0.3256
Epoch 6/200
13/13 [==============================] - 0s 10ms/step - loss: 0.0890 - accuracy: 0.4987 - val_loss: 0.0993 - val_accuracy: 0.3721
Epoch 7/200
13/13 [==============================] - 0s 9ms/step - loss: 0.0821 - accuracy: 0.4935 - val_loss: 0.0943 - val_accuracy: 0.3256
Epoch 8/20

In [216]:
a = tf.constant(X)
tf.math.l2_normalize(a).numpy()

array([[[ 0.        ,  0.        ,  0.        , ...,  0.00552828,
         -0.00659141, -0.00677609],
        [ 0.00757559,  0.        ,  0.        , ..., -0.00238965,
         -0.01646883, -0.01451381],
        [ 0.        ,  0.00757559,  0.        , ..., -0.00472415,
         -0.00212759, -0.00230309],
        [ 0.        ,  0.        ,  0.00757559, ...,  0.00232412,
          0.00348978,  0.00299009]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.00921513,
          0.00537546,  0.00040839],
        [ 0.00757559,  0.        ,  0.        , ...,  0.0002729 ,
          0.00560671,  0.00508392],
        [ 0.        ,  0.00757559,  0.        , ..., -0.00203002,
          0.00129858,  0.00159386],
        [ 0.        ,  0.        ,  0.00757559, ...,  0.01397219,
         -0.00468563, -0.00910651]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.01151   ,
         -0.00425536, -0.00547394],
        [ 0.00757559,  0.        ,  0.        , ...,  0.00533579,
         -0.

In [75]:
from sklearn.model_selection import train_test_split

trainX, testX, trainy, testy =  train_test_split(X, Y, test_size = 0.4,random_state=42) 

evaluate_model(trainX, trainy, testX, testy)

ValueError: in user code:

    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/tahaalnufaili/

In [ ]:
testX.shape, trainy.shape

((1249, 1), (834, 57))